<a href="https://colab.research.google.com/github/dnjswngo/artificial-intelligence-programming/blob/master/230102.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 언어 모델
언어모델(language model, lm)은 주어진 문장 이후에 올 단어의 확률을 예측하는 모델

확률을 예측할 때는 모든 단어에 대해서 확률을 매기며 가장 확률이 높은 단어가 모델이 예측하는 다음 단어가 됨> 주어진 문장을 보고 그 다음에 올 가장 그럴듯한 단어를 예측하는 모델

잘 학습된 언어 모델만으로도 언어 모델에서 예측한 다음 단어로 새로운 sequence를 생성하고 다음 단어를 예측하는 것을 반복하며 새로운 문장을 생성해낼 수 있음

## 사전 학습 언어모델
언어 모델을 학습시킬 때 corpus만 있으면 기계적으로 시퀀스 및 다음 단어를 만들어서 학습데이터로 활용 가능함. 즉 별도의 수작업을 통한 라벨링이 필요하지 않음

따라서 대규모의 데이터셋에 대해 수행하기 좋은 특성이 있으며 또한 언어 모델이라는 것은 문맥을 어느 정도 고려하여 다음 단어를 예측하여야 하므로 해당 언어에 관한 전반적인 지식이 축적된 모델이라고 볼 수 있음

전이학습처럼 자연어 처리에서도 언어 모델을 다른 세부적인 작업을 수행하기 위한 모델의 가중치 초기화 용도로 사용하기 시작 했고 이 떄 사용되는 언어모델을 pre-trained LM이라고 함

OpenAI에서 자연어 및 시퀀스 처리에 강점을 보이는 transformer모델을 활용한 사전학습 언어모델을 선보임. 약자로 GPT(generative ore-trained transformer)

## Attention mechanism 
attention이란 예측값을 계산하기 위해 입력을 차모할 때 입력값을 그대로 보는 것이 아니라 각각의 입력값(자연어>토큰, 이미지>블록)에 대한 중요도를 매번 새로 계산하여 현재 시점에 가장 적절한 부분에 집중해서 입력을 살펴보는 방식. 계산된 attention 기반으로 입력값*attention이 신경망에 입력됨

## Transformer
기계 번역, 텍스트 생성 모델 등 널리 쓰이는 인코더-디코더 구조를 갖고 있으며 특징으로는 순환 신경망과 같은 순환 연산 구조 없이 attention 및 전결합층만 활용해 높은 성능을 달성

인코더를 통해 압축된 정보는 디코더의 매 layer에서 참조되어 결과 sequence를 만들어내는 데 기여

Attetion 및 출력값을 계산하기 위해서는 attention을 계산하고자 하는 현재 시점의 토큰인 query, attention을 계산할 때의 전체 대상값은 key 그리고 구해진 attention이 곱해질 입력값 value가 필요

입력 sequence 자체에 대해 attention을 수행하면 self-attention이라고 함

transformer에서는 임베딩을 그대로 쓰는 것이 아니라 q, k, v를 계산하는 가중치를 따로 두어서 한 임베딩에서 서로 다른 qkv를 계산후 사용힘



In [1]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 27.1 MB/s 
     |████████████████████████████████| 182 kB 71.9 MB/s 
     |████████████████████████████████| 7.6 MB 66.0 MB/s 


In [2]:
!wget http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
!tar -xf aclImdb_v1.tar.gz

--2023-01-03 09:05:38--  http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
Resolving ai.stanford.edu (ai.stanford.edu)... 171.64.68.10
Connecting to ai.stanford.edu (ai.stanford.edu)|171.64.68.10|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 84125825 (80M) [application/x-gzip]
Saving to: ‘aclImdb_v1.tar.gz’

aclImdb_v1.tar.gz   100%[===================>]  80.23M  15.5MB/s    in 8.2s    

2023-01-03 09:05:46 (9.81 MB/s) - ‘aclImdb_v1.tar.gz’ saved [84125825/84125825]



In [3]:
from pathlib import Path

def read_imdb_split(split_dir):
  split_dir=Path(split_dir)
  texts=[]
  labels=[]
  for label_dir in ["pos","neg"]:
    for text_file in (split_dir/label_dir).iterdir():
      texts.append(text_file.read_text())
      labels.append(0 if label_dir is "neg" else 1)
    return texts, labels

train_texts, train_labels=read_imdb_split('aclImdb/train')
test_texts, test_labels=read_imdb_split('aclImdb/test')

<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
<ipython-input-3-cfc587a2b3c1>:10: SyntaxWarning: "is" with a literal. Did you mean "=="?
  labels.append(0 if label_dir is "neg" else 1)


In [4]:
from transformers import DistilBertTokenizerFast #단어를 더 쪼개 
tokenizer=DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')
train_encodings=tokenizer(train_texts, truncation=True, padding=True)
test_encodings=tokenizer(test_texts, truncation=True, padding=True) #max_length 길이 안맞을 때 자르거나 padding을 붙이기


Downloading:   0%|          | 0.00/28.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/466k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/483 [00:00<?, ?B/s]

In [5]:
#모델 입력 위한 데이터셋
import torch as t

class IMDbDataset(t.utils.data.Dataset):
  def __init__(self, encodings, labels):
    self.encodings=encodings
    self.labels=labels
  
  def __getitem__(self, idx):
    item={key: t.tensor(val[idx]) for key, val in self.encodings.items()}
    item['labels']=t.tensor(self.labels[idx])
    return item

  def __len__(self):
    return len(self.labels)

train_dataset=IMDbDataset(train_encodings, train_labels)
test_dataset=IMDbDataset(test_encodings, test_labels)

In [6]:
#하이퍼파라미터랑 pre-trained LM 로딩
from transformers import DistilBertForSequenceClassification, Trainer, TrainingArguments
from sklearn.metrics import accuracy_score

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    num_train_epochs=1,              # total number of training epochs
    per_device_train_batch_size=25,  # batch size per device during training
    per_device_eval_batch_size=50,   # batch size for evaluation
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=250,
)

model=DistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
"""
trainer에서 evaluate 메서드만 호출하면 loss, 예측 속도 측면에서의 성능만 기록됨
추가적인 성능 지표 사용 위해 만든 객체
"""
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    
    acc = accuracy_score(labels, preds)    
    return {
        'accuracy': acc
    }
  
trainer=Trainer(
    model=model,                    #the instantiated, transformers model to be trained
    args=training_args,             #training arguments, defined above
    train_dataset=train_dataset,    #training dataset
    eval_dataset=test_dataset,      #evaluation dataset
    compute_metrics=compute_metrics #additional evaluation metrics
)

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertForSequenceClassification: ['vocab_layer_norm.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_transform.weight']
- This IS expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'pre_classifier.weight', 'classi

In [7]:
trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 12500
  Num Epochs = 1
  Instantaneous batch size per device = 25
  Total train batch size (w. parallel, distributed & accumulation) = 25
  Gradient Accumulation steps = 1
  Total optimization steps = 500
  Number of trainable parameters = 66955010


Step,Training Loss
250,0.053100
500,0.000100


Saving model checkpoint to ./results/checkpoint-500
Configuration saved in ./results/checkpoint-500/config.json
Model weights saved in ./results/checkpoint-500/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=500, training_loss=0.02655998445302248, metrics={'train_runtime': 548.4301, 'train_samples_per_second': 22.792, 'train_steps_per_second': 0.912, 'total_flos': 1655842483200000.0, 'train_loss': 0.02655998445302248, 'epoch': 1.0})

In [8]:
metrics=trainer.evaluate(eval_dataset=test_dataset)

***** Running Evaluation *****
  Num examples = 12500
  Batch size = 50


In [9]:
metrics

{'eval_loss': 3.2266620110021904e-05,
 'eval_accuracy': 1.0,
 'eval_runtime': 186.6044,
 'eval_samples_per_second': 66.987,
 'eval_steps_per_second': 1.34,
 'epoch': 1.0}

In [11]:
"""
transformers에서는 pipline이라는 high-level api로 학습된 모델 데려올 수 있음
모델을 직접 pytorch로 가져올 수도 있는데 복잡해진대
"""
from transformers import pipeline

nlp=pipeline("sentiment-analysis")

result=nlp("I hate you")[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}.")

result=nlp('I love you')[0]
print(f"label: {result['label']}, with score: {round(result['score'], 4)}.")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/bfdd146ea2b6807255b73527f1327ca12b6ed5c4/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading configuration file config

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/bfdd146ea2b6807255b73527f1327ca12b6ed5c4/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForSequenceClassification.

All the weights of DistilBertForSequenceClassification were initialized from the model checkpoint at distilbert-base-uncased-finetuned-sst-2-english.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForSequenceClassification for predictions without further training.


Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/bfdd146ea2b6807255b73527f1327ca12b6ed5c4/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "finetuning_task": "sst-2",
  "hidden_dim": 3072,
  "id2label": {
    "0": "NEGATIVE",
    "1": "POSITIVE"
  },
  "initializer_range": 0.02,
  "label2id": {
    "NEGATIVE": 0,
    "POSITIVE": 1
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}



Downloading:   0%|          | 0.00/232k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/bfdd146ea2b6807255b73527f1327ca12b6ed5c4/vocab.txt
loading file tokenizer.json from cache at None
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/bfdd146ea2b6807255b73527f1327ca12b6ed5c4/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-uncased-finetuned-sst-2-english/snapshots/bfdd146ea2b6807255b73527f1327ca12b6ed5c4/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased-finetuned-sst-2-english",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  

label: NEGATIVE, with score: 0.9991.
label: POSITIVE, with score: 0.9999.


In [12]:
#question answering
nlp=pipeline("question-answering")

context= r"""
Extractive Question Answering is the task of extracting an answer from a text given a question. An example of a
question answering dataset is the SQuAD dataset, which is entirely based on that task. If you would like to fine-tune
a model on a SQuAD task, you may leverage the examples/question-answering/run_squad.py script.
"""
result=nlp(question="What is extractive question answering?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

result=nlp(question="What is a good example of a question answer dataset?", context=context)
print(f"Answer: '{result['answer']}', score: {round(result['score'], 4)}, start: {result['start']}, end: {result['end']}")

No model was supplied, defaulted to distilbert-base-cased-distilled-squad and revision 626af31 (https://huggingface.co/distilbert-base-cased-distilled-squad).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/473 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": true,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 28996
}

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/config.json
Model config Disti

Downloading:   0%|          | 0.00/261M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/pytorch_model.bin
All model checkpoint weights were used when initializing DistilBertForQuestionAnswering.

All the weights of DistilBertForQuestionAnswering were initialized from the model checkpoint at distilbert-base-cased-distilled-squad.
If your task is similar to the task the model of the checkpoint was trained on, you can already use DistilBertForQuestionAnswering for predictions without further training.


...

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBertForQuestionAnswering"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "output_past": true,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": true,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 28996
}



Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/436k [00:00<?, ?B/s]

loading file vocab.txt from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/vocab.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/tokenizer_config.json
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--distilbert-base-cased-distilled-squad/snapshots/a8440a9ab1b3d7a603df3349516078434abade1e/config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-cased-distilled-squad",
  "activation": "gelu",
  "architectures": [
    "DistilBe

Answer: 'the task of extracting an answer from a text given a question', score: 0.6226, start: 34, end: 95
Answer: 'SQuAD dataset', score: 0.4861, start: 147, end: 160


In [13]:
text_generator=pipeline("text-generation") #문장 생성하기
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=False))
#max~는 문장 최대 단어수, do~는 확률 높은거만 고를지 샘플링할건지

No model was supplied, defaulted to gpt2 and revision 6c0e608 (https://huggingface.co/gpt2).
Using a pipeline without specifying a model name and revision in production is not recommended.


Downloading:   0%|          | 0.00/665 [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_activation": null,
  "summary_first_dropout": 0.1,
  "summary_proj_to_labels": true,
  "summary_type": "cls_index",
  "summary_use_proj": true,
  "task_specific_params": {
    "text-generation": {
      "do_sample": true,
      "max_length": 50
    }
  },
  "transformers_vers

Downloading:   0%|          | 0.00/548M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at gpt2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_range": 0.02,
  "lay

Downloading:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/456k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

loading file vocab.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/vocab.json
loading file merges.txt from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/merges.txt
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--gpt2/snapshots/e7da7f221d5bf496a48136c0cd264e630fe9fcc8/config.json
Model config GPT2Config {
  "_name_or_path": "gpt2",
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 50256,
  "embd_pdrop": 0.1,
  "eos_token_id": 50256,
  "initializer_ran

[{'generated_text': 'As far as I am concerned, I will be the first to admit that I am not a fan of the idea of a "free market." I think that the idea of a free market is a bit of a stretch. I think that the idea'}]


In [14]:
print(text_generator("As far as I am concerned, I will", max_length=50, do_sample=True, top_k=3))
#상위 3퍼센트 단어들중 !

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[{'generated_text': 'As far as I am concerned, I will never be able to get my hands on a new game.\n\nI have a few questions. I am going to ask you to take the time to answer these questions. I am not going to give'}]


In [15]:
text_generator=pipeline("text-generation", model='skt/kogpt2-base-v2')
print(text_generator("요즘 부쩍 날씨가 추워졌습니다", max_length=50, do_sample=False))

Downloading:   0%|          | 0.00/1.00k [00:00<?, ?B/s]

loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "license": "CC-BY-NC-SA 4.0",
  "model_type": "gpt2",
  "n_ctx": 1024,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 1024,
  "pad_token_id": 3,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,

Downloading:   0%|          | 0.00/513M [00:00<?, ?B/s]

loading weights file pytorch_model.bin from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/pytorch_model.bin
All model checkpoint weights were used when initializing GPT2LMHeadModel.

All the weights of GPT2LMHeadModel were initialized from the model checkpoint at skt/kogpt2-base-v2.
If your task is similar to the task the model of the checkpoint was trained on, you can already use GPT2LMHeadModel for predictions without further training.
Could not locate the tokenizer configuration file, will try to use the model config instead.
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(mad

Downloading:   0%|          | 0.00/2.83M [00:00<?, ?B/s]

loading file vocab.json from cache at None
loading file merges.txt from cache at None
loading file tokenizer.json from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/tokenizer.json
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at None
loading configuration file config.json from cache at /root/.cache/huggingface/hub/models--skt--kogpt2-base-v2/snapshots/d0c0df48bf2b2c9350dd855021a5b216f560c0c7/config.json
Model config GPT2Config {
  "_name_or_path": "skt/kogpt2-base-v2",
  "_num_labels": 1,
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "author": "Heewon Jeon(madjakarta@gmail.com)",
  "bos_token_id": 0,
  "created_date": "2021-04-28",
  "embd_pdrop": 0.1,
  "eos_token_id": 1,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "i

[{'generated_text': '요즘 부쩍 날씨가 추워졌습니다만 오늘은 어제보다 더 춥겠습니다.\n서울의 아침기온이 영하 4도까지 떨어지겠고요.\n한낮에도 영하권에 머물겠습니다.\n서울의 낮기온이 영하 2'}]


In [21]:
print(text_generator("빨리 연구주제를 정해야돼요", max_length=50, do_sample=True, top_k=3))

[{'generated_text': '빨리 연구주제를 정해야돼요!\n그리고 이게 뭐예요?\n이건 뭐죠?\n그런데 왜 그걸 왜 안 하세요?\n그냥 그냥 연구주제를 정하는 거예요?\n이거 왜 안 하세요?\n왜 안'}]
